In [1]:
import datajoint as dj
import numpy as np
import os
import time
import trimesh

In [2]:
pinky = dj.create_virtual_module("pinky","microns_pinky")
schema = dj.schema("microns_pinky")

Connecting celiib@10.28.0.34:3306


In [10]:
def filter_verts_and_faces(verts,faces,verts_label,triangles_label):
    verts = verts.tolist()
    faces = faces.tolist()
    verts_label = verts_label.tolist()
    triangles_label = triangles_label.tolist()
    
    verts_keep = []
    faces_keep = []
    verts_lookup = {}
    
    for i,ver in enumerate(verts_label):
        verts_keep.append(verts[ver])
        verts_lookup[ver] = i
    
    #generate the new face labels
    for fac in triangles_label:
        faces_with_verts = faces[fac]
        new_tuple = []
        for v in faces_with_verts:
            new_tuple.append(verts_lookup[v])
        
        faces_keep.append(new_tuple)
    #check that the new verts and faces to return are same length as the indices
    """if len(triangles_label) != len(faces_keep) or len(verts_label) != len(verts_keep):
        print("ERROR THE FILTERED LABELS ARE NOT THE SAME SIZE AS THE INDICES LISTS!")"""
     
    return verts_keep,faces_keep

In [25]:
window = 10
size_threshold = 2000
(pinky.LeftoverCompartmentFinal.LeftoverComponentFinal() & 
     "n_triangle_indices >= " + str(size_threshold - window) & 
     "n_triangle_indices <= " + str(size_threshold + window) & 
     [dict(compartment_type=comp) for comp in ["Apical","Basal","Dendrite","Oblique"]])

segmentation segmentation id,segment_id segment id unique within each Segmentation,decimation_ratio,"compartment_type Basal, Apical, spine head, etc.",component_index Which sub-compartment of a certain label this is.,n_vertex_indices,n_triangle_indices,vertex_indices preserved indices of each vertex of this sub-compartment,triangle_indices preserved indices of each triangle of this sub-compartment
3,648518346349493260,0.35,Dendrite,4,998,1990,=BLOB=,=BLOB=
3,648518346349496405,0.35,Basal,22,999,1997,=BLOB=,=BLOB=
3,648518346349498286,0.35,Apical,1,1003,1996,=BLOB=,=BLOB=
3,648518346349498566,0.35,Basal,11,1005,2004,=BLOB=,=BLOB=
3,648518346349505144,0.35,Basal,2,1006,1996,=BLOB=,=BLOB=
3,648518346349506425,0.35,Basal,29,1005,2003,=BLOB=,=BLOB=
3,648518346349506684,0.35,Basal,7,995,1990,=BLOB=,=BLOB=


In [29]:
#3	648518346349491311	0.35	Basal	8  maybe a little too small
#3	648518346349506748	0.35	Basal	20 maybe good size
#3	648518346349494577	0.35	Basal	5
#3	648518346349506684	0.35	Basal	7	 #2000 level definitely good
#3	648518346349498566	0.35	Basal	11	
segmentation=3
segment_id = 648518346349498566
decimation_ratio = 0.35
compartment_type = "Basal"
component_index = 11



example_key = dict(segmentation=segmentation,
            segment_id=segment_id,
            decimation_ratio=decimation_ratio,
            compartment_type=compartment_type,
            component_index=component_index)



original_start_time = time.time()  

#get the full mesh
verts,faces = (pinky.ExcitatoryLeftoverMeshes & dict(segmentation=segmentation, segment_id=segment_id)).fetch1("vertices","triangles")

#get the indices
vertex_indices,triangle_indices = (pinky.LeftoverCompartmentFinal.LeftoverComponentFinal 
                                   & example_key).fetch1("vertex_indices","triangle_indices")

verts_keep,faces_keep = filter_verts_and_faces(verts,faces,vertex_indices,triangle_indices)

test_mesh = trimesh.Trimesh(verts_keep,faces_keep,process=False)
test_mesh.show()

/usr/local/lib/python3.6/dist-packages/IPython/core/display.py:689: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


In [ ]:
"""
Conclusion: Decided to do pieces up to 2000 mesh faces to be consevative
and make sure that spines don't mess up the data

But going to run them on anything above 500
and can then just restrict the overlay code
if they don't look good

"""